<a href="https://colab.research.google.com/github/castillosebastian/NLU_legal_domain/blob/master/NLU_Stanza_%26_Semantica_Vectorial_en_Textos_Legales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantica Vectorial en Textos Legales: Exploración

In [1]:
# Libraries, tools and thanks 
import bs4
import nltk
import numpy
import pandas as pd
import io
import requests
from google.colab import drive
import numpy as np 
import json

In [2]:
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [3]:
%cd  'drive/My Drive/Colab Notebooks/data'

/content/drive/My Drive/Colab Notebooks/data


In [4]:
%ls

corpus_fallosmetdat.json  tbdoctrina.json  tbmetdat.json
corpus_textosfallos.json  tbfallos.json


# Lectura de tablas de datos primarios

In [5]:
with open('tbfallos.json', 'r') as myfile:
    data1=myfile.read()
# parse file
tbfallos = json.loads(data1)
tbfallos = pd.DataFrame.from_dict(tbfallos)

In [6]:
tbfallos['textos_fallo'][1]

'#TEXTO COMPLETO#A C U E R D O#En la ciudad de La Plata, a 15 de mayo de 2019, habiéndose establecido, de conformidad con lo dispuesto en el Acuerdo 2078, que deberá observarse el siguiente orden de votación: doctores Soria, de Lázzari, Genoud, Kogan, se reúnen los señores Jueces de la Suprema Corte de Justicia en acuerdo ordinario para pronunciar sentencia definitiva en la causa C. 122.321, "F., R.. Determinación de la capacidad jurídica".#A N T E C E D E N T E S#La Sala I de la Cámara de Apelación en lo Civil y Comercial de San Isidro confirmó el fallo de primera instancia que, en los autos sobre determinación de la capacidad jurídica del señor R. F., resolvió aprobar la rendición de cuentas efectuada por la Curaduría Oficial de Alienados (v. fs. 689 y 660, respectivamente).#Se interpuso, por la titular de la Asesoría de Incapaces n° 2 departamental, recurso extraordinario de inaplicabilidad de ley (v. fs. 705/711).#Oído el señor Procurador General, dictada la providencia de autos y 

In [7]:
with open('tbmetdat.json', 'r') as myfile:
    data1=myfile.read()
# parse file
tbmetdat = json.loads(data1)
tbmetdat = pd.DataFrame.from_dict(tbmetdat)

In [8]:
tbmetdat[tbmetdat.nro_causa == 122321]

,nro_causa,fres,textos_raw_len,materia,tipo_fallo,tribunal_emisor,caratula,caratula_pub,magistrados_votantes,tribunal_origen,bnro,magistrados,sumario,voces
1,122321,2019-05-15,160,CIVIL Y COMERCIAL,Sentencia Definitiva,SUPREMA CORTE DE JUSTICIA DE LA PROVINCIA (SCBA),"Figueredo, Rafael s/ Determinación de la capac...","F. ,R. s/ Determinación de la capacidad jurídica",Soria-de Lázzari-Genoud-Kogan,CAMARA DE APELACION EN LO CIVIL Y COMERCIAL SA...,158.0,Soria - de Lázzari - Genoud - Kogan.,\nSentencia-Fundamentación. Rendición de cuen...,"RENDICION DE CUENTAS,APROBACION,SENTENCIA,FUND..."


In [ ]:
tbmetdat.loc[1]

In [10]:
tbmetdat.loc[1].sumario

' \nSentencia-Fundamentación. Rendición de cuentas-Aprobación. \nLa Suprema Corte de Justicia de la Provincia de Buenos Aires revocó la sentencia que tubo por \naprobada  la rendición  de  cuentas  presentada  por  el  Curador  Oficial  en  el  marco  del  proceso de \ndeterminación de la capacidad, considerando la falta de fundamentación suficiente del fallo y de \nadecuación a los hechos probados en la causa. '

In [11]:
with open('tbdoctrina.json', 'r') as myfile:
    data1=myfile.read()
# parse file
tbdoctrina = json.loads(data1)
tbdoctrina = pd.DataFrame.from_dict(tbdoctrina)

In [12]:
tbdoctrina.head()

,nro_causa,doctrina_nro,doctrina
0,122514,1,El recurso de inaplicabilidad de ley no consti...
1,122514,2,Los juicios de divorcio en trámite al momento ...
2,122514,3,Cuando en la causa sobre divorcio las partes n...
3,122514,4,El artículo 7 del nuevo Código Civil y Comerci...
4,122514,5,La aplicación inmediata de las normas se disti...


In [13]:
tbdoctrina[tbdoctrina.nro_causa == 122321].doctrina

10    Debe ser revocado el fallo que, como único sus...
Name: doctrina, dtype: object

# Procesamiento con Stanza (StanfordNLP)

- https://github.com/stanfordnlp/stanza

In [ ]:
! pip install stanza

In [ ]:
import stanza
stanza.download('es') # download English model
nlp = stanza.Pipeline('es') # initialize Spanish neural pipeline
doc = nlp("Alberto Fernandez es presidente de Argetina") # run annotation over a sentence

In [16]:
doc

[
  [
    {
      "id": 1,
      "text": "Alberto",
      "lemma": "Alberto",
      "upos": "PROPN",
      "xpos": "PROPN",
      "head": 4,
      "deprel": "nsubj",
      "misc": "start_char=0|end_char=7",
      "ner": "B-PER"
    },
    {
      "id": 2,
      "text": "Fernandez",
      "lemma": "Fernandez",
      "upos": "PROPN",
      "xpos": "PROPN",
      "head": 1,
      "deprel": "flat",
      "misc": "start_char=8|end_char=17",
      "ner": "E-PER"
    },
    {
      "id": 3,
      "text": "es",
      "lemma": "ser",
      "upos": "AUX",
      "xpos": "AUX",
      "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin",
      "head": 4,
      "deprel": "cop",
      "misc": "start_char=18|end_char=20",
      "ner": "O"
    },
    {
      "id": 4,
      "text": "presidente",
      "lemma": "presidente",
      "upos": "NOUN",
      "xpos": "NOUN",
      "feats": "Gender=Masc|Number=Sing",
      "head": 0,
      "deprel": "root",
      "misc": "start_char=21|end_char=31",


# Tokenizacion y Segmentacion de sentencias

## Método por REGEX

In [ ]:
import re
def split_into_sentences(text):       
    text = text.replace("#","<stop>")    
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences
split_into_sentences(tbfallos['textos_fallo'][1])

## Método Stanza

In [17]:
text = tbfallos['textos_fallo'][1]
text = text.replace("#","\n\n")

In [ ]:
nlp = stanza.Pipeline(lang='es', processors='tokenize', tokenize_no_ssplit=True)
doc = nlp(text)
for i, sentence in enumerate(doc.sentences):
    print(f'====== Sentence {i+1} tokens =======')
    print(*[f'id: {token.id}\ttext: {token.text}' for token in sentence.tokens], sep='\n')

In [ ]:
doc.sentences[11]

# POS y Ufeat

In [ ]:
# nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos')
# print(*[f'word: {word.text}\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}' for sent in doc.sentences for word in sent.words], sep='\n')

# Entidades

In [19]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,ner')
doc = nlp(text)
print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in doc.ents], sep='\n')

2020-10-24 12:45:56 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| ner       | conll02 |

2020-10-24 12:45:56 INFO: Use device: cpu
2020-10-24 12:45:56 INFO: Loading: tokenize
2020-10-24 12:45:56 INFO: Loading: ner
2020-10-24 12:45:58 INFO: Done loading processors!


entity: TEXTO COMPLETO	type: MISC
entity: La Plata	type: LOC
entity: Acuerdo 2078	type: MISC
entity: Soria	type: PER
entity: Lázzari	type: PER
entity: Genoud	type: PER
entity: Kogan	type: PER
entity: Suprema Corte de Justicia	type: ORG
entity: C.	type: MISC
entity: Sala I de la Cámara de Apelación	type: ORG
entity: Civil	type: MISC
entity: Comercial de San Isidro	type: ORG
entity: señor R. F.	type: PER
entity: Curaduría Oficial de Alienados	type: ORG
entity: Asesoría de Incapaces n° 2	type: ORG
entity: Procurador General	type: PER
entity: Suprema Corte	type: ORG
entity: Juez doctor Soria	type: PER
entity: señor R. F.	type: PER
entity: Juzgado de Familia n° 4	type: ORG
entity: San Isidro	type: LOC
entity: Curaduría Oficial de Alienados de San Isidro	type: ORG
entity: Zárate-Campana	type: ORG
entity: Cuenta de Autos	type: MISC
entity: Cuenta Fiscal de Curaduría	type: ORG
entity: señor F.	type: PER
entity: Asesora de Incapaces	type: ORG
entity: Daniela Elizabeth Ferreyra	type: PER
entity:

## POS y Mofología

In [20]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos')
doc = nlp(text)
print(*[f'word: {word.text}\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}' for sent in doc.sentences for word in sent.words], sep='\n')


2020-10-24 13:00:17 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |

2020-10-24 13:00:17 INFO: Use device: cpu
2020-10-24 13:00:17 INFO: Loading: tokenize
2020-10-24 13:00:17 INFO: Loading: mwt
2020-10-24 13:00:17 INFO: Loading: pos
2020-10-24 13:00:19 INFO: Done loading processors!


word: TEXTO	upos: NOUN	xpos: NOUN	feats: Gender=Masc|Number=Sing
word: COMPLETO	upos: ADJ	xpos: ADJ	feats: Gender=Masc|Number=Sing
word: A	upos: ADP	xpos: ADP	feats: AdpType=Prep
word: C	upos: NOUN	xpos: NOUN	feats: _
word: U	upos: CCONJ	xpos: CCONJ	feats: _
word: E	upos: CCONJ	xpos: CCONJ	feats: _
word: R	upos: PROPN	xpos: PROPN	feats: _
word: D	upos: PROPN	xpos: PROPN	feats: _
word: O	upos: CCONJ	xpos: CCONJ	feats: _
word: En	upos: ADP	xpos: ADP	feats: AdpType=Prep
word: la	upos: DET	xpos: DET	feats: Definite=Def|Gender=Fem|Number=Sing|PronType=Art
word: ciudad	upos: NOUN	xpos: NOUN	feats: Gender=Fem|Number=Sing
word: de	upos: ADP	xpos: ADP	feats: AdpType=Prep
word: La	upos: DET	xpos: DET	feats: Definite=Def|Gender=Fem|Number=Sing|PronType=Art
word: Plata	upos: PROPN	xpos: PROPN	feats: _
word: ,	upos: PUNCT	xpos: PUNCT	feats: PunctType=Comm
word: a	upos: ADP	xpos: ADP	feats: AdpType=Prep
word: 15	upos: NUM	xpos: NUM	feats: NumForm=Digit|NumType=Card
word: de	upos: ADP	xpos: ADP	feats

# Dependencia Sintáctica

In [21]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma,depparse')
doc = nlp('Alberto Fernandez es el presidente de Argentina')
print(*[f'id: {word.id}\tword: {word.text}\thead id: {word.head}\thead: {sent.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for sent in doc.sentences for word in sent.words], sep='\n')

2020-10-24 13:12:34 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |

2020-10-24 13:12:34 INFO: Use device: cpu
2020-10-24 13:12:34 INFO: Loading: tokenize
2020-10-24 13:12:34 INFO: Loading: mwt
2020-10-24 13:12:34 INFO: Loading: pos
2020-10-24 13:12:35 INFO: Loading: lemma
2020-10-24 13:12:35 INFO: Loading: depparse
2020-10-24 13:12:36 INFO: Done loading processors!


id: 1	word: Alberto	head id: 5	head: presidente	deprel: nsubj
id: 2	word: Fernandez	head id: 1	head: Alberto	deprel: flat
id: 3	word: es	head id: 5	head: presidente	deprel: cop
id: 4	word: el	head id: 5	head: presidente	deprel: det
id: 5	word: presidente	head id: 0	head: root	deprel: root
id: 6	word: de	head id: 7	head: Argentina	deprel: case
id: 7	word: Argentina	head id: 5	head: presidente	deprel: nmod


In [23]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma,depparse')
doc = nlp(text)
print(*[f'id: {word.id}\tword: {word.text}\thead id: {word.head}\thead: {sent.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for sent in doc.sentences for word in sent.words], sep='\n')

2020-10-24 13:14:39 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |

2020-10-24 13:14:39 INFO: Use device: cpu
2020-10-24 13:14:39 INFO: Loading: tokenize
2020-10-24 13:14:39 INFO: Loading: mwt
2020-10-24 13:14:39 INFO: Loading: pos
2020-10-24 13:14:40 INFO: Loading: lemma
2020-10-24 13:14:40 INFO: Loading: depparse
2020-10-24 13:14:41 INFO: Done loading processors!


id: 1	word: TEXTO	head id: 0	head: root	deprel: root
id: 2	word: COMPLETO	head id: 1	head: TEXTO	deprel: amod
id: 1	word: A	head id: 2	head: C	deprel: case
id: 2	word: C	head id: 0	head: root	deprel: root
id: 3	word: U	head id: 5	head: R	deprel: cc
id: 4	word: E	head id: 5	head: R	deprel: cc
id: 5	word: R	head id: 2	head: C	deprel: conj
id: 6	word: D	head id: 5	head: R	deprel: flat
id: 7	word: O	head id: 2	head: C	deprel: punct
id: 1	word: En	head id: 3	head: ciudad	deprel: case
id: 2	word: la	head id: 3	head: ciudad	deprel: det
id: 3	word: ciudad	head id: 17	head: establecido	deprel: obl
id: 4	word: de	head id: 6	head: Plata	deprel: case
id: 5	word: La	head id: 6	head: Plata	deprel: det
id: 6	word: Plata	head id: 3	head: ciudad	deprel: nmod
id: 7	word: ,	head id: 9	head: 15	deprel: punct
id: 8	word: a	head id: 9	head: 15	deprel: case
id: 9	word: 15	head id: 3	head: ciudad	deprel: nmod
id: 10	word: de	head id: 11	head: mayo	deprel: case
id: 11	word: mayo	head id: 9	head: 15	deprel: com